# rag walkthrough（细讲版）

## 目标与先修
- 目标：理解 `report -> chunks -> retrieval -> answer payload -> final answer` 的 RAG 全链路。
- 先修：了解文本切分、Top-k 检索、相似度评分基本概念。


## 流程总览
1. 读取原始报告文本。
2. 进行 chunk 切分。
3. 执行 Top-k 检索并观察得分。
4. 生成 answer payload 并调用完整 demo。


### 环境检查（在线模块）
- 本步做什么：检查 `DEEPSEEK_API_KEY` 是否存在。
- 为什么这样做：`rag` 依赖在线 LLM API；缺 key 时必须显式失败，避免误以为流程成功。
- 输入：环境变量 `DEEPSEEK_API_KEY`。
- 输出：通过则继续执行；失败则抛出 `RuntimeError`。
- 观察点：失败信息要明确提示“这是配置问题，不是业务逻辑问题”。


In [ ]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY", "")
if not api_key:
    raise RuntimeError("缺少 DEEPSEEK_API_KEY：在线 walkthrough 不能继续执行。")

print("环境检查通过：已检测到 DEEPSEEK_API_KEY")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 导入函数并设置查询
- 本步做什么：导入读文件、切分、检索、回答构造和 demo 函数。
- 为什么这样做：把 RAG 每个阶段单独暴露出来，便于观察中间态。
- 输入：`query` 与 `top_k`。
- 输出：可复用的函数集合和参数。
- 观察点：同一 query 在不同 `top_k` 下结果会变化。


In [ ]:
from component import (
    DEFAULT_QUERY,
    chunk_report,
    load_report_text,
    request_answer_payload,
    retrieve_top_k,
    run_demo,
)

query = DEFAULT_QUERY
top_k = 3
show_kv("调试参数", {"query": query, "top_k": top_k})


### Step 2: 读取原始报告文本
- 本步做什么：调用 `load_report_text()`。
- 为什么这样做：检索质量先取决于语料质量。
- 输入：默认报告路径。
- 输出：完整报告字符串。
- 观察点：先看文本长度和前几百字符，确认读取正常。


In [ ]:
report_text = load_report_text()
show_kv("报告概览", {"length": len(report_text), "preview": report_text[:220]})


### Step 3: 文本切分为 chunks
- 本步做什么：调用 `chunk_report(report_text)`。
- 为什么这样做：RAG 检索通常在 chunk 级别进行，而不是全量长文本。
- 输入：`report_text`。
- 输出：chunk 列表。
- 观察点：查看 chunk 数量与首个 chunk 内容。


In [ ]:
chunks = chunk_report(report_text)
print("chunk count:", len(chunks))
show_kv("首个 chunk 预览", {"chunk_0": chunks[0][:240] if chunks else ""})


### Step 4: Top-k 检索并解释分数
- 本步做什么：调用 `retrieve_top_k(chunks, query, top_k=top_k)`。
- 为什么这样做：检索结果决定后续回答引用哪些上下文。
- 输入：`chunks`、`query`、`top_k`。
- 输出：包含 `chunk_id/score/text` 的命中列表。
- 观察点：分数越高通常相关性越强；检查是否命中 query 关键主题。


In [ ]:
hits = retrieve_top_k(chunks, query, top_k=top_k)
show_json(hits)

print()
print("检索分数解读:")
for hit in hits:
    print(f"chunk_id={hit['chunk_id']}, score={round(hit['score'], 4)}")


### Step 5: 构造 answer payload
- 本步做什么：调用 `request_answer_payload(query, hits)`。
- 为什么这样做：把检索命中整理成模型可消费的回答上下文。
- 输入：`query` 与 `hits`。
- 输出：结构化 answer payload。
- 观察点：payload 中必须体现 query 与引用片段。


In [ ]:
answer_payload = request_answer_payload(query, hits)
show_json(answer_payload)


## 端到端结果

### Step 6: 运行 run_demo
- 本步做什么：执行 `run_demo(query=query, top_k=top_k)` 获取最终回答与 trace。
- 为什么这样做：确认分步结果和端到端流程一致。
- 输入：`query`、`top_k`。
- 输出：`demo_result`。
- 观察点：trace 必须能对应读取、切分、检索、生成各阶段。


In [ ]:
demo_result = run_demo(query=query, top_k=top_k)
show_json(demo_result)

print()
print("FINAL ANSWER:")
print(demo_result["final_answer"])


## 常见错误
- 缺少 `DEEPSEEK_API_KEY`：在线生成阶段无法执行。
- `hits` 为空：query 与语料不匹配，或切分/检索参数不合适。
- final answer 质量低：优先检查 Top-k 命中内容是否相关。

## 总结
- RAG 调试不能只看最终回答，必须先确认检索命中是否正确。
- 关键观测链路：`chunks` 数量 -> `hits` 得分 -> answer payload -> trace。
